In [502]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

In [503]:
# file_name_str = r"..\..\Sray\Analysis\AU\AU_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\CN\CN_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\HK\HK_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\MY\MY_Bloomberg_fin_data_portfolio.xlsx"
file_name_str = r"..\Sray\Analysis\SG\SG_Bloomberg_fin_data_portfolio.xlsx"

# indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'historical_market_cap'
# indicator = 'roa'
indicator = 'vol_180d'
## indicator = 'return_com_eqy'


file_name = Path(file_name_str).stem

lo_quantile_folder = "..\..\Sray\Analysis_bottom\\" + file_name[:2] + "\\" + indicator
lo_reporting_path = lo_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_low.xlsx"
lo_fin_file = pd.ExcelFile(lo_reporting_path)
lo_findata_sheet = pd.read_excel(
    lo_fin_file, sheet_name="main", header=0)

lo_porfolio_selection = pd.DataFrame(lo_findata_sheet)

hi_quantile_folder = "..\..\Sray\Analysis_top\\" + file_name[:2] + "\\" + indicator
hi_reporting_path = hi_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_second_highest.xlsx"
hi_fin_file = pd.ExcelFile(hi_reporting_path)
hi_findata_sheet = pd.read_excel(
    hi_fin_file, sheet_name="main", header=0)

hi_porfolio_selection = pd.DataFrame(hi_findata_sheet)


In [504]:
lo_porfolio_selection.columns

Index(['year', 'no of firms', 'no of firms(bottom 20)', 'rm_r_y1', 'rm_r_y2',
       'eq_wg_ret_p', 'fc_wg_ret_p', 'lo_eq_wg_r_p', 'lo_eq_wg_r_p_y1',
       'eq_r_alpha_y1', 'lo_eq_wg_r_p_y2', 'eq_r_alpha_y2', 'lo_fc_wg_r_p',
       'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'lo_fc_wg_r_p_y2', 'fc_r_alpha_y2',
       'lo_eq_wg_rc_p', 'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2',
       'eq_rc_alpha_y2', 'lo_fc_wg_rc_p', 'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1',
       'lo_fc_wg_rc_p_y2', 'fc_rc_alpha_y2'],
      dtype='object')

In [505]:
hi_porfolio_selection.columns

Index(['year', 'no of firms', 'no of firms(top 20)', 'rm_r_y1', 'rm_r_y2',
       'eq_wg_ret_p', 'fc_wg_ret_p', 'eq_wg_r_p', 'eq_wg_r_p_y1',
       'eq_r_alpha_y1', 'eq_wg_r_p_y2', 'eq_r_alpha_y2', 'fc_wg_r_p',
       'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_wg_r_p_y2', 'fc_r_alpha_y2',
       'eq_wg_rc_p', 'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2',
       'eq_rc_alpha_y2', 'fc_wg_rc_p', 'fc_wg_rc_p_y1', 'fc_rc_alpha_y1',
       'fc_wg_rc_p_y2', 'fc_rc_alpha_y2'],
      dtype='object')

In [506]:
if indicator == 'historical_market_cap' or indicator == 'vol_180d':
    hi_porfolio_selection['eq_r_alpha3_y1'] =  lo_porfolio_selection['lo_eq_wg_r_p_y1']- \
    hi_porfolio_selection['eq_wg_r_p_y1']

    hi_porfolio_selection['fc_r_alpha3_y1'] = lo_porfolio_selection['lo_fc_wg_r_p_y1'] - \
        hi_porfolio_selection['fc_wg_r_p_y1']

    hi_porfolio_selection['eq_rc_alpha3_y1'] =  lo_porfolio_selection['lo_eq_wg_rc_p_y1']- \
        hi_porfolio_selection['eq_wg_rc_p_y1']

    hi_porfolio_selection['fc_rc_alpha3_y1'] =  lo_porfolio_selection['lo_fc_wg_rc_p_y1'] - \
        hi_porfolio_selection['fc_wg_rc_p_y1']

else:
    hi_porfolio_selection['eq_r_alpha3_y1'] = hi_porfolio_selection['eq_wg_r_p_y1'] - \
        lo_porfolio_selection['lo_eq_wg_r_p_y1']

    hi_porfolio_selection['fc_r_alpha3_y1'] = hi_porfolio_selection['fc_wg_r_p_y1'] - \
        lo_porfolio_selection['lo_fc_wg_r_p_y1']

    hi_porfolio_selection['eq_rc_alpha3_y1'] = hi_porfolio_selection['eq_wg_rc_p_y1'] - \
        lo_porfolio_selection['lo_eq_wg_rc_p_y1']

    hi_porfolio_selection['fc_rc_alpha3_y1'] = hi_porfolio_selection['fc_wg_rc_p_y1'] - \
        lo_porfolio_selection['lo_fc_wg_rc_p_y1']

hi_porfolio_selection.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
5,2022,105.0,25.000000,NaN,NaN,-0.027002,-0.026119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,105.0,25.000000,NaN,NaN,0.064316,0.064435,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,average,105.0,24.571429,-0.133933,0.549419,-0.031383,-0.032965,-0.067191,-0.806293,-0.341095,...,0.240112,0.002022,0.024268,0.158202,0.783188,0.233769,0.346673,0.395139,0.004324,0.015499
8,t-score,NaN,NaN,-0.686899,0.216696,-0.721676,-0.755934,-0.858432,-0.858432,-0.750189,...,1.528295,-0.400842,-0.400842,3.426249,0.466503,1.505637,-1.049894,-0.829498,0.112544,0.122404
9,p-value,NaN,NaN,0.541480,0.842345,0.522656,0.504629,0.453770,0.453770,0.507616,...,0.223901,0.715377,0.715377,0.041652,0.672642,0.229235,-0.301393,-0.254653,0.077178,0.083388


In [507]:
portfolio_selection_variances_t_test = hi_porfolio_selection.copy()
portfolio_selection_variances_t_test = hi_porfolio_selection.iloc[:-3]
portfolio_selection_variances_t_test.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
2,2019,105.0,25.0,-0.075984,-0.073393,0.089619,0.085737,0.073049,0.876593,0.952577,...,0.225204,0.011845,0.142140,0.218124,0.151776,0.225169,-0.821070,-0.695182,0.012290,0.012191
3,2020,105.0,25.0,-1.215864,-0.722509,-0.195610,-0.191964,-0.198660,-2.383923,-1.168059,...,0.011954,-0.099076,-1.188908,0.026956,-0.714070,0.008439,1.396385,1.365988,0.380677,0.386106
4,2021,105.0,25.0,1.252343,2.291070,0.067811,0.066338,0.063567,0.762809,-0.489534,...,0.790571,0.123921,1.487050,0.234707,3.062829,0.771759,-0.940403,-0.913493,-0.555646,-0.532979
5,2022,105.0,25.0,NaN,NaN,-0.027002,-0.026119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,105.0,25.0,NaN,NaN,0.064316,0.064435,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [508]:
SG_esg_int_mean = portfolio_selection_variances_t_test.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean.insert(0, 'year', 'Average', True)
SG_esg_int_mean = pd.concat(
    [portfolio_selection_variances_t_test, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
3,2020,105.0,25.000000,-1.215864,-0.722509,-0.195610,-0.191964,-0.198660,-2.383923,-1.168059,...,0.011954,-0.099076,-1.188908,0.026956,-0.714070,0.008439,1.396385,1.365988,0.380677,0.386106
4,2021,105.0,25.000000,1.252343,2.291070,0.067811,0.066338,0.063567,0.762809,-0.489534,...,0.790571,0.123921,1.487050,0.234707,3.062829,0.771759,-0.940403,-0.913493,-0.555646,-0.532979
5,2022,105.0,25.000000,NaN,NaN,-0.027002,-0.026119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,105.0,25.000000,NaN,NaN,0.064316,0.064435,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Average,105.0,24.571429,-0.133933,0.549419,-0.031383,-0.032965,-0.067191,-0.806293,-0.341095,...,0.240112,0.002022,0.024268,0.158202,0.783188,0.233769,0.346673,0.395139,0.004324,0.015499


In [509]:
SG_res_dict_t_test = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, portfolio_selection_variances_t_test.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=portfolio_selection_variances_t_test.dropna()[portfolio_selection_variances_t_test.columns[column]], popmean=0)
    temp_dict = {portfolio_selection_variances_t_test.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict_t_test = {**SG_res_dict_t_test, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict_t_test[i]
                                 for i in SG_res_dict_t_test.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict


,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,eq_wg_r_p_y2,eq_r_alpha_y2,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
0,t-score,-0.686899,0.216696,-0.721676,-0.755934,-0.858432,-0.858432,-0.750189,0.224296,-0.035561,...,1.528295,-0.400842,-0.400842,3.426249,0.466503,1.505637,0.486378,0.564324,0.534263,0.574507
1,p-value,0.541480,0.842345,0.522656,0.504629,0.453770,0.453770,0.507616,0.836934,0.973866,...,0.223901,0.715377,0.715377,0.041652,0.672642,0.229235,0.660016,0.611973,0.630216,0.605876


In [510]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df], axis=0)
SG_esg_int.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2,eq_r_alpha3_y1,fc_r_alpha3_y1,eq_rc_alpha3_y1,fc_rc_alpha3_y1
5,2022,105.0,25.000000,NaN,NaN,-0.027002,-0.026119,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,105.0,25.000000,NaN,NaN,0.064316,0.064435,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Average,105.0,24.571429,-0.133933,0.549419,-0.031383,-0.032965,-0.067191,-0.806293,-0.341095,...,0.240112,0.002022,0.024268,0.158202,0.783188,0.233769,0.346673,0.395139,0.004324,0.015499
0,t-score,NaN,NaN,-0.686899,0.216696,-0.721676,-0.755934,-0.858432,-0.858432,-0.750189,...,1.528295,-0.400842,-0.400842,3.426249,0.466503,1.505637,0.486378,0.564324,0.534263,0.574507
1,p-value,NaN,NaN,0.541480,0.842345,0.522656,0.504629,0.453770,0.453770,0.507616,...,0.223901,0.715377,0.715377,0.041652,0.672642,0.229235,0.660016,0.611973,0.630216,0.605876


In [511]:
export_file_path = "..\..\Sray\Analysis\\" + file_name[:2] + "\\" + indicator + "\\" + file_name[:2]+ '_' + indicator + "_alpha3_int_reporting.xlsx"
SG_esg_int.to_excel(export_file_path , sheet_name="main", index=False)